## Import pakietów

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix

In [ ]:
# Wybieramy CPU lub GPU w zależności od dostępności.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Tworzenie zbioru danych

Teraz rozdzielamy obrazy na validacyjne, testowe i treningowe.

In [2]:
!pip install split-folders[full]

In [9]:
!ls /kaggle/input/five-flowers/original-jpegs

daisy  dandelion  roses  sunflowers  tulips


In [16]:
!splitfolders --seed 42 --output /kaggle/working/data --ratio .7 .2 .1 -- /kaggle/input/five-flowers/original-jpegs

Copying files: 3670 files [00:04, 767.95 files/s] 


In [ ]:
IMAGE_SIZE = (64, 64) # To wydaje sie optymalny rozmiar do tego problemu.

# Transformacje
transformations = transforms.Compose([
    # Tu wstaw transformacje
    # Transformacje są dostępne na https://pytorch.org/vision/stable/transforms.html
    # Przynajmniej potrzebujemy normalizacji wielkości losowej rotacji i flip. I zamiany na tensor (sklauje do <0, 1>)
])

In [ ]:
TRAIN_PATH = "/kaggle/working/data/train"
VAL_PATH = "/kaggle/working/data/val"
TEST_PATH = "/kaggle/working/data/test"

# Dataset do validacji i do trenowania
# Stwórz z https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html#torchvision.datasets.ImageFolder

Teraz tworzymy obiekty do ładowania danych.

In [ ]:
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Model

In [ ]:
model = nn.Sequential([
    # Tu wstaw warstwy modelu.
    # Ekstraktor cech powinien mieć kilka bloków składających się z warstw konwolucyjnych, max pooling, ReLU i batch normalization.
    # Na koniec potrzebujemy flatten aby zamienić obraz na wektor i potem liniowego klasyfikatora.
    # Ilość wyjść to 5. Ilość wejść do warstwy liniowej to H*W*C wyjścia z ekstraktora cech. Można to ustalić eksperymentalnie poprzez wprowadzenie losowego wejścia do sieci.
    # Zobacz torch.randn.
    # Komponenty są dostępne na https://pytorch.org/docs/stable/nn.html
])

model = model.to(device)

## Optymalizator

In [ ]:
LR = 1e-3 # Jeśli model nie będzie się trenował to pewnie trzeba to zmniejszyć.
WEIGHT_DECAY = 0.0 # Waga do regularyzacji z L2
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

## Funkcja strat i pętal trenowania

In [ ]:
criterion = nn.CrossEntropyLoss()

loss_train = []
loss_val = []

N_EPOCHS = 10 # Ile przejść przez zbiór danych.
for epoch in range(N_EPOCHS):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        preds = model(x)
        loss = criterion(preds, y)

        # Obliczanie gradientów
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_train.append(float(loss.cpu()))
    
    
    ## Napisz kod do ewaluacji sieci na zbiorze valdacyjnym.
    ## Oblicz średni loss i accuracy.
    ## Predykcję można ustalić przy użyciu argmax w torch,

In [ ]:
# Plot loss curves
plt.plot(loss_train, label="train")
plt.plot(loss_val, label="val")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.legend()

## Ewaluacja 

In [ ]:
model = model.eval() # Wyłącz aktualizację parametrów w normalizacji partii.
## Napisz kod do ewaluacji modelu na zbiorze testowym. Oblicz accuracy. Ustal confusion matrix przy użyciu 
## https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

## Notatka: kod do predykcji powinien być w bloku with torch.no_grad(): żeby wyłączyć automatyczne różniczkowanie.

## Eksport modelu do ONNX

In [ ]:
dummy_input = torch.randn(1, 3, X, Y) # Ustaw rozmiar który był użyty na początku
torch.onnx.export(model, dummy_input, "model.onnx")